# Neutral Ensembleizer
This notebook handles running neutral ensemble analysis. It takes in shapefiles that have already been run through `Ensemble Prep.ipynb`
<span style="color:red">**CHANGE THE FILE NAMES FOR EXPORTS**</span>.

### Imports

In [ ]:
# For working with graphs
import networkx as nx
# For making plots
import matplotlib.pyplot as plt
import numpy as np

# Needed for gerrchain
import gerrychain   
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
import geopandas as gpd
import pandas as pd

# For making plots
import matplotlib.pyplot as plt
import numpy as np

# Needed for gerrchain
import gerrychain   
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
import geopandas as gpd
import pandas as pd

#For maup.assign to add current plans
import maup

import random


### Making the Ensemble

Import our .shp that we prepared in `Ensemble Prep.ipynb`

In [ ]:
state_gdf = gpd.read_file("Montana/MT_ready_4_ensemble/MT_ready_4_ensemble.shp")

Add a current districting plan to place it on the boxplots.

In [ ]:
current_districting_plan = gpd.read_file("Montana/Final_Senate_Districts_2023 2023-02-20/Final_SenateDistricts_Errate_02_23_2023 2023-02-23.shp")
current_districting_plan=current_districting_plan.to_crs(4269)
precincts_to_districts_assignment = maup.assign(state_gdf.geometry, current_districting_plan.geometry)
state_gdf['district'] = precincts_to_districts_assignment

In [ ]:
state_graph = Graph.from_geodataframe(state_gdf) #Convert gdf into a graph

In [ ]:
tot_pop = sum([state_graph.nodes()[v]['P1_001N'] for v in state_graph.nodes()])
num_dist = 50
ideal_pop = tot_pop/num_dist
pop_tolerance = 0.1 #At times had to increase this beyond what was desirable in order for recursive_tree_part to work
initial_plan = recursive_tree_part(state_graph, range(num_dist), ideal_pop, 'P1_001N', pop_tolerance, 100)

In [ ]:
#Set up partition object

initial_partition = Partition(
    state_graph, # dual graph
    assignment = initial_plan, # initial districting plan
    updaters = { 
        "our cut edges": cut_edges, 
        "district population": Tally("P1_001N", alias = "district population"), 
        "district NAPOP": Tally("P1_005N", alias = "district NAPOP"),
        "district NAPOP_MIXED": Tally("P1_012N", alias = "district NAPOP_MIXED"),
        "R Votes": Tally("G20PRERTRU", alias = "R Votes"), 
        "D Votes": Tally("G20PREDBID", alias = "D Votes")
    }
) 

In [ ]:
rw_proposal = partial(recom, ## how you choose a next districting plan
                      pop_col = "P1_001N", 
                      pop_target = ideal_pop, ## What the target/ideal population is for each district 
                      epsilon = pop_tolerance,  ## how far from ideal population you can deviate
                      node_repeats = 10 ## number of times to repeat bipartition.  Can increase if you get a BipartitionWarning
                      )

In [ ]:
population_constraint = constraints.within_percent_of_ideal_population(
    initial_partition, 
    pop_tolerance, 
    pop_key = "district population"
    )

Setting up the chain itself using the components we've set up.

In [ ]:
our_random_walk = MarkovChain(
    proposal = rw_proposal, 
    constraints = [population_constraint],
    accept = always_accept, # accepts every proposed plan that meets population criteria
    initial_state = initial_partition, 
    total_steps = 10000
)

### Running the Chain

In [ ]:
cutedge_ensemble = []
nativemaj_ensemble = []
nativemixedmaj_ensemble = []
native_mixed_and_single_maj_ensemble = []
d_ensemble = []

for part in our_random_walk:
    # Add cutedges to cutedges ensemble
    cutedge_ensemble.append(len(part["our cut edges"]))
    
    # Calculate number of native-majority districts 
    # Add to ensemble
    native_this_step = []
    for i in range(num_dist):
        na_perc = part["district NAPOP"][i]/part["district population"][i]
        native_this_step.append(na_perc)
    native_this_step.sort()
    nativemaj_ensemble.append(native_this_step)
    # Calculate number of white-native-mixed-majority districts 
    # Add to ensemble
    maj_native_mixed_this_step = []
    for i in range(num_dist):
        na_mixed_perc = part["district NAPOP_MIXED"][i]/part["district population"][i]
        maj_native_mixed_this_step.append(na_mixed_perc)
    maj_native_mixed_this_step.sort()
    nativemixedmaj_ensemble.append(maj_native_mixed_this_step)
    # Calculate number of districts where there is a majority white-native-mixed and native-single combined
    # Add to ensemble
    num_maj_native_mixed_and_single_this_step = []
    for i in range(num_dist):
        native_mixed_and_single_perc = (part["district NAPOP_MIXED"][i]+part["district NAPOP"][i])/part["district population"][i]
        num_maj_native_mixed_and_single_this_step.append(native_mixed_and_single_perc)
    num_maj_native_mixed_and_single_this_step.sort()
    native_mixed_and_single_maj_ensemble.append(num_maj_native_mixed_and_single_this_step)
    # Calculate number of districts with more Democratic votes than Republican votes
    d = 0
    for i in range(num_dist):
        if part["R Votes"][i] < part["D Votes"][i]: 
            d = d + 1
    d_ensemble.append(d)


print(cutedge_ensemble)
print(nativemaj_ensemble)
print(nativemixedmaj_ensemble)
print(d_ensemble)

### Creating Plots from our Data

In [ ]:

dist_napop = [0] * num_dist
dist_pop = [0] * num_dist
dist_frac = [0] * num_dist
a = np.array(nativemaj_ensemble)

for node in state_graph.nodes(): 
    dist = int(state_graph.nodes()[node]["district"]) - 1 # Because districts are indexed starting at 1, not 0
    dist_napop[dist] = dist_napop[dist] + state_graph.nodes()[node]["P1_005N"]
    dist_pop[dist] = dist_pop[dist] + state_graph.nodes()[node]["P1_001N"]

#Used for scatter plot for current plan
for dist in range(1, num_dist):
    dist_frac[dist] = (dist_napop[dist]/dist_pop[dist])

dist_frac.sort()


from pathlib import Path
Path("Plots/MT/ensemble/2/native_single/2/").mkdir(parents=True, exist_ok=True)
# plot poitns on top of boxplot
plt.figure()
plt.axis([19.5, 31, 0, 0.7]) #useful for massive plots
plt.boxplot(a, showfliers=False)
plt.scatter(x = range(1, num_dist + 1), y = dist_frac, color = "blue" )
plt.savefig('Plots/MT/ensemble/2/native_single/zoomedin2') 



plt.figure(figsize = (20,7))
plt.boxplot(a)
plt.scatter(x = range(1, num_dist + 1), y = dist_frac, color = "blue" )
plt.savefig('Plots/MT/ensemble/2/native_single/zoomedout2') 






In [ ]:
a = np.array(native_mixed_and_single_maj_ensemble)



dist_napop = [0] * num_dist
dist_pop = [0] * num_dist
dist_frac = [0] * num_dist


for node in state_graph.nodes(): 
    dist = int(state_graph.nodes()[node]["district"]) - 1 # Because districts are indexed starting at 1, not 0
    dist_napop[dist] = dist_napop[dist] + state_graph.nodes()[node]["P1_005N"] + state_graph.nodes()[node]["P1_012N"]
    dist_pop[dist] = dist_pop[dist] + state_graph.nodes()[node]["P1_001N"]

for dist in range(1, num_dist):
    dist_frac[dist] = (dist_napop[dist]/dist_pop[dist])

dist_frac.sort()


from pathlib import Path
Path("Plots/MT/ensemble/2/native_single+mixed").mkdir(parents=True, exist_ok=True)

# plot poitns on top of boxplot
plt.figure()
plt.axis([19.5, 31, 0, 0.7]) #useful for massive plots
plt.boxplot(a, showfliers=False)
plt.scatter(x = range(1, num_dist + 1), y = dist_frac, color = "blue" )
plt.savefig("Plots/MT/ensemble/2/native_single+mixed/zoomedin2")




plt.figure(figsize = (20,7))
#plt.axis([88.5, 102, 0, 0.6]) #usful for massive plots
plt.boxplot(a)
plt.scatter(x = range(1, num_dist + 1), y = dist_frac, color = "blue" )
plt.savefig("Plots/MT/ensemble/2/native_single+mixed/zoomedout2")


plt.figure()
plt.show()

In [ ]:
plt.figure()
plt.hist(d_ensemble, align = 'left', bins='auto')
plt.savefig("Plots/MT/ensemble/2/d_ensemble2")

In [ ]:

plt.figure()
plt.hist(cutedge_ensemble, align = 'left', bins='auto')
plt.savefig("Plots/MT/ensemble/2/cutedges")